<a href="https://colab.research.google.com/github/Suhas-S63/PLLM_Lab_Exercises/blob/main/Fine_Tuning_with_LLM_model_(Activity).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers
! pip install datasets
! pip install sentencepiece
! pip install rouge_score
! pip install wandb
! pip install transformers[torch]
! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24

In [2]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    try:
        import wandb
        wandb.login()
    except Exception as e:
        print(f"Failed to log into WandB: {e}")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
# Importing necessary libraries
import torch
import numpy as np
import datasets
from transformers import (
   AutoModelForSeq2SeqLM,
   AutoTokenizer,
   Seq2SeqTrainingArguments,
   Seq2SeqTrainer,
   DataCollatorForSeq2Seq,
)
from tabulate import tabulate
import nltk
from datetime import datetime

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Model name
pretrained_model_name = "facebook/bart-large"

try:
    model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name)
    # tokenizer
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
    print(f"Successfully loaded model and tokenizer for {pretrained_model_name}")
except Exception as e:
    print(f"Error loading model or tokenizer: {e}")

# tokenization parameters
encoder_max_len = 512
decoder_max_len = 128


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Successfully loaded model and tokenizer for facebook/bart-large


In [8]:
# dataset
import pandas as pd
geographical_data_df = pd.read_csv('/content/Geographicaldata.csv', encoding='latin-1', sep=',',on_bad_lines='skip') #The encoding='latin-1' argument is used to handle invalid characters appropriately.

In [9]:
import pandas as pd
from datasets import Dataset
# Convert DataFrame to Dataset
geographical_data = Dataset.from_pandas(geographical_data_df)
def flatten_example(example):
   return {
       "CityDescription": example["City Description"],
       "FeatureExtraction": example["Feature Extraction"],
   }
def filter_samples(example):
   CityDescription = []
   FeatureExtraction = []
   for desc, feat in zip(example["City Description"], example["Feature Extraction"]):
       if len(desc) > 0:
           CityDescription.append(desc)
           FeatureExtraction.append(feat)
   return {"CityDescription": CityDescription, "FeatureExtraction": FeatureExtraction}

# transformations to the dataset
geographical_data = geographical_data.map(flatten_example)
geographical_data = geographical_data.map(filter_samples, batched=True)
# Split the dataset into train and validation sets
train_data_txt, validation_data_txt = geographical_data.train_test_split(test_size=0.1).values()

Map:   0%|          | 0/299 [00:00<?, ? examples/s]

Map:   0%|          | 0/299 [00:00<?, ? examples/s]

### Preprocessing Text and using this for the trainning and validation data preparation

In [11]:
def preprocess_batch(batch, tokenizer, max_source_length, max_target_length):
   src, tgt = batch["CityDescription"], batch["FeatureExtraction"]
   src_tokenized = tokenizer(
       src, padding="max_length", truncation=True, max_length=max_source_length
   )
   tgt_tokenized = tokenizer(
       tgt, padding="max_length", truncation=True, max_length=max_target_length
   )
   batch = {k: v for k, v in src_tokenized.items()}
   # Ignore padding in the loss
   batch["labels"] = [
       [-100 if token == tokenizer.pad_token_id else token for token in l]
       for l in tgt_tokenized["input_ids"]
   ]
   return batch

#### Training Data Preparation

In [12]:
train_data = train_data_txt.map(
   lambda batch: preprocess_batch(
       batch, tokenizer, encoder_max_len, decoder_max_len
   ),
   batched=True,
   remove_columns=train_data_txt.column_names,
)


Map:   0%|          | 0/269 [00:00<?, ? examples/s]

#### VAlidation DAta Preparation

In [13]:
validation_data = validation_data_txt.map(
   lambda batch: preprocess_batch(
       batch, tokenizer, encoder_max_len, decoder_max_len
   ),
   batched=True,
   remove_columns=validation_data_txt.column_names,
)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

## Building the Model

In [14]:
import numpy as np
from datasets import load_metric
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer
nltk.download("punkt", quiet=True)
metric = load_metric("rouge")
def postprocess_text(preds, labels):
   preds = [pred.strip() for pred in preds]
   labels = [label.strip() for label in labels]
   # rougeLSum expects newline after each sentence
   preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
   labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]
   return preds, labels

<ipython-input-14-13c42cffecff>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge") #Recall-Oriented Understudy for Gisting Evaluation


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


## Function for Evaluation Metric Calculation with ROUGE

In [15]:
def calculate_metrics(eval_preds):
   preds, labels = eval_preds
   if isinstance(preds, tuple):
       preds = preds[0]
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   # -100 in the labels as we can't decode them.
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
   # Some simple post-processing
   decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
   result = metric.compute(
       predictions=decoded_preds, references=decoded_labels, use_stemmer=True
   )
   # Extract a few results from ROUGE
   result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

   prediction_lens = [
       np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
   ]
   result["gen_len"] = np.mean(prediction_lens)
   result = {k: round(v, 4) for k, v in result.items()}
   return result


### Training the model now wiht the custom dataset and the configurations

In [17]:
training_args = Seq2SeqTrainingArguments(
   output_dir="results",
   num_train_epochs=1,
   do_train=True,
   do_eval=True,
   per_device_train_batch_size=4,
   per_device_eval_batch_size=4,
   warmup_steps=500,
   weight_decay=0.1,
   label_smoothing_factor=0.1,
   predict_with_generate=True,
   logging_dir="logs",
   logging_steps=50,
   save_total_limit=3,
)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   data_collator=data_collator,
   train_dataset=train_data,
   eval_dataset=validation_data,
   tokenizer=tokenizer,
   compute_metrics=calculate_metrics,
)

### Evaluation before training

In [18]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: suhas-s (suhas-s-christ-university). Use `wandb login --relogin` to force relogin


{'eval_loss': 5.975897789001465,
 'eval_rouge1': 5.5534,
 'eval_rouge2': 1.8637,
 'eval_rougeL': 5.1112,
 'eval_rougeLsum': 5.1018,
 'eval_gen_len': 20.0,
 'eval_runtime': 10.2375,
 'eval_samples_per_second': 2.93,
 'eval_steps_per_second': 0.781}

Training the Model now

In [19]:
trainer.train()


Step,Training Loss
50,4.301200


TrainOutput(global_step=68, training_loss=3.9027666765100815, metrics={'train_runtime': 69.8273, 'train_samples_per_second': 3.852, 'train_steps_per_second': 0.974, 'total_flos': 291475568984064.0, 'train_loss': 3.9027666765100815, 'epoch': 1.0})

#### Evaluation After Training

In [20]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 2.5822131633758545,
 'eval_rouge1': 62.3853,
 'eval_rouge2': 45.9788,
 'eval_rougeL': 59.0372,
 'eval_rougeLsum': 59.1747,
 'eval_gen_len': 12.3333,
 'eval_runtime': 7.2575,
 'eval_samples_per_second': 4.134,
 'eval_steps_per_second': 1.102,
 'epoch': 1.0}

### Generating Summary


In [21]:
def generate_summary(test_data, model):
   inputs = tokenizer(
       test_data["CityDescription"],
       padding="max_length",
       truncation=True,
       max_length=encoder_max_len,
       return_tensors="pt",
   )
   input_ids = inputs.input_ids.to(model.device)
   attention_mask = inputs.attention_mask.to(model.device)
   outputs = model.generate(input_ids, attention_mask=attention_mask)
   output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
   return outputs, output_str
model_before_fine_tuning = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name)
test_samples = validation_data_txt.select(range(10))
features_before_fine_tuning = generate_summary(test_samples, model_before_fine_tuning)[1]
features_after_fine_tuning = generate_summary(test_samples, model)[1]

In [22]:
print("\nTarget Feature:\n")
print(
   tabulate(list(enumerate(test_samples["FeatureExtraction"])), headers=["Id", "Target Feature"])
)
print("\nCity Description:\n")
print(tabulate(list(enumerate(test_samples["CityDescription"])), headers=["Id", "City Description"]))



Target Feature:

  Id  Target Feature
----  --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
   0  religious importance, pilgrimage site
   1  Rich cultural heritage, historical monuments.
   2  Major technology and business hub, historical landmarks like Charminar and Golconda Fort.
   3  Famous Ongole cattle breed, Handloom industry (production of Ongole cotton sarees)
   4  Natural beauty including Sundarbans mangrove forest and Ganges delta, rich cultural heritage reflected in folk music, dance, and festivals, and textile and handloom industries.
   5  Features: Located on the banks of the Wular Lake, known for scenic landscapes including the Arin Wildlife Sanctuary.
   6  Coal mining industry, Industrial significance
   7  Important industrial center, Educational institutions
   8  administrative importance, cultural heritage
   9  agricultural significa